<a href="https://colab.research.google.com/github/cicl-iscl/LeWiDi_SemEval2023/blob/main/Notebooks/bert_multilingual_last_layer_dataset_upload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notes

ConvAbuse is just the dictionary as a string lol

In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Set up wandb

In [2]:
import wandb
config = dict (
  architecture = "bert_arabic_english",
  epochs = 1,
  batch_size = 4
)

run = wandb.init(
  project="capture-disagreement",
  notes="test_run",
  #name = "test_run",
  config=config,
  save_code = True,
  tags = ["bert_arabic_english", "weighted_loss", "balanced_loss", "only_last_layer"],
  #magic = True
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: eolian (capture_disagreement). Use `wandb login --relogin` to force relogin


# Load data

In [3]:
import pandas as pd
import os
import ast

artifact_dataset = run.use_artifact('capture_disagreement/capture-disagreement/dataset_combined_train:v0', type='dataset')
artifact_dir = artifact_dataset.download()
df_all = pd.read_csv(os.path.join(artifact_dir, "dataset_combined_train.csv"))
df_all['soft_label'] = df_all['soft_label'].apply(lambda x: ast.literal_eval(x))

wandb:   1 of 1 files downloaded.  


# Dataset

In [4]:
from datasets import Dataset
dataset = Dataset.from_pandas(df_all[["text", "hard_label", "soft_label"]])

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("lanwuwei/GigaBERT-v4-Arabic-and-English", do_lower_case=True)

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=240)#This is the longest that roberta tokenizer would go, dunno

tokenized_datasets = dataset.map(tokenize_function, batched=False)

  0%|          | 0/10431 [00:00<?, ?ex/s]

In [7]:
tokenized_datasets.set_format("torch")#not sure if neccessary


In [8]:
from torch.utils.data import DataLoader
batch_size = 4

In [9]:
from transformers import AutoModel
model = AutoModel.from_pretrained("bert-base-multilingual-cased")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
import torch
from tqdm import tqdm_notebook
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
m = model.to(device)

In [11]:
train_dataloader = DataLoader(tokenized_datasets, shuffle=True, batch_size=batch_size)

In [12]:
import torch

# 1. Iterate through the dataset and save the outputs of the model
embeddings = []
datasets = []
with torch.no_grad():
  for i, batch in enumerate(tqdm_notebook(train_dataloader, 0)):
      inputs, attention_mask, hard_labels, soft_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device), batch["soft_label"]
      outputs = model(input_ids=inputs, attention_mask = attention_mask)#NllLoss
      embeddings.append(outputs.last_hidden_state)
      if len(embeddings) == 500:
        embeddings_combined = torch.cat(embeddings, dim=0)
        dataset_embeddings_train = torch.utils.data.TensorDataset(embeddings_combined)
        datasets.append(dataset_embeddings_train)
        embeddings = []

if len(embeddings) > 0:
  embeddings_combined = torch.cat(embeddings, dim=0)
  dataset_embeddings_train = torch.utils.data.TensorDataset(embeddings_combined)
  datasets.append(dataset_embeddings_train)
  embeddings = []

dataset_embeddings_train = torch.utils.data.ConcatDataset(datasets)

<ipython-input-12-c43d9919e641>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, batch in enumerate(tqdm_notebook(train_dataloader, 0)):


  0%|          | 0/2608 [00:00<?, ?it/s]

In [ ]:

# Save the dataset to disk
torch.save(dataset_embeddings_train, 'dataset_embeddings_train.pt')
del dataset_embeddings_train

# Load the dataset from disk
#dataset_embeddings_train = torch.load('dataset_embeddings_train.pt')


In [5]:
#import torch
#dataset_embeddings_train = torch.load('dataset_embeddings_train.pt')


In [6]:
#dataset_embeddings_train[0]

(tensor([[ 0.3764, -0.5129,  0.3775,  ..., -0.1143, -0.0547, -0.2479],
         [ 0.4659, -0.2307,  0.0994,  ...,  0.7505,  0.2975, -0.2568],
         [ 0.5035,  0.1338,  0.6321,  ...,  0.6203,  0.2793, -0.1642],
         ...,
         [ 0.3159, -0.1183,  0.3395,  ...,  0.2533,  0.2331, -0.1664],
         [ 0.3111, -0.2354,  0.5080,  ...,  0.1643,  0.4167, -0.1107],
         [ 0.5519, -0.2302,  0.4622,  ...,  0.2034,  0.4145, -0.1540]],
        device='cuda:0'),)

In [7]:
artifact_dataset_embeddings = wandb.Artifact(name='bert_multilingual_last_layer_dataset', type='dataset')
artifact_dataset_embeddings.add_file(local_path='dataset_embeddings_train.pt')
run.log_artifact(artifact_dataset_embeddings)

In [8]:
run.finish()